In [6]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid, Amazon, Coauthor, CitationFull
from torch_geometric.loader import DataLoader
import numpy as np

In [ ]:
# GCN 기반 GNN 모델 정의
class GCN(torch.nn.Moudle):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)
        
    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

# 분위수 손실 함수 정의
class QuantileLoss(torch.nn.Module):
    def __init__(self):
        super(QuantileLoss, self).__init__()
        
    def forward(self, yhat, y, tau):
        diff = yhat - y
        mask = (diff.ge(0))

In [7]:
# GNN 모델 정의
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels,
                             cached=True, normalize=True)
        self.conv2 = GCNConv(hidden_channels, out_channels,
                             cached=True, normalize=True)
        
    def forward(self, x, edge_index, edge_weight=None, return_h=False):
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

In [3]:
# 데이터셋 로드
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]

Processing...
Done!


In [4]:
# 데이터셋 분할: Train/ Calibration / Test
num_nodes = data.x.shape[0]
indices = np.arange(num_nodes)
np.random.shuffle(indices)

train_ratio, calib_ratio = 0.6, 0.2 # 20% test
train_size = int(num_nodes * train_ratio)
calib_size = int(num_nodes * calib_ratio)

train_idx = indices[:train_size]
calib_idx = indices[train_size:train_size + calib_size]
test_idx = indices[train_size + calib_size:]

In [5]:
# new 마스크 설정
data.train_mask = torch.zeros(num_nodes, dtype=torch.bool)
data.train_mask[train_idx] = True

data.calib_mask = torch.zeros(num_nodes, dtype=torch.bool)  # Calibration Mask 추가
data.calib_mask[calib_idx] = True

data.test_mask = torch.zeros(num_nodes, dtype=torch.bool)
data.test_mask[test_idx] = True

In [9]:
# 모델 훈련 함수
def train_model(model, data, optimizer, epochs=200):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = F.mse_loss(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()
    return model

# Conformal Prediction 보정 함수 (Calibration Set 사용)
def conformal_prediction_calibrate(model, data, alpha=0.1):
    model.eval()
    with torch.no_grad():
        logits = model(data.x, data.edge_index).numpy()
        labels = data.y.numpy()
        scores = np.abs(logits - labels)  # 비순응 점수 계산 (회귀)
    
    # Calibration 데이터만 사용하여 신뢰구간 보정
    qhat = np.quantile(scores[data.calib_mask], 1 - alpha)
    return qhat

# Conformal Prediction 적용 함수
def conformal_prediction_predict(model, data, qhat):
    model.eval()
    with torch.no_grad():
        logits = model(data.x, data.edge_index).numpy()
        lower_bound = logits - qhat
        upper_bound = logits + qhat
    return lower_bound, upper_bound

In [10]:
# Conformal Prediction 수행
qhat = conformal_prediction_calibrate(model, data, alpha=0.1)
lower_bound, upper_bound = conformal_prediction_predict(model, data, qhat)

# 결과 출력
print("Calibration Quantile Threshold:", qhat)
print("Lower Bound:", lower_bound)
print("Upper Bound:", upper_bound)


NameError: name 'model' is not defined